<a href="https://colab.research.google.com/github/ChyavanShenoy/AAI-500-Final-Project/blob/main/Project_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final project AAI-500

Members:   
- Chyavan Shenoy
- Koelgeet Kaur
- Asad Salah

# Imports

In [258]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
# from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
import xgboost as xgb
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score
import joblib
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt

# Loading Dataset

In [259]:
df = pd.read_csv('AQI_dataset.csv')
df.head()

,Country,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category,lat,lng
0,Russian Federation,Praskoveya,51,Moderate,1,Good,36,Good,0,Good,51,Moderate,44.7444,44.2031
1,Brazil,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good,-5.2900,-44.4900
2,Brazil,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good,-11.2958,-41.9869
3,Italy,Priolo Gargallo,66,Moderate,1,Good,39,Good,2,Good,66,Moderate,37.1667,15.1833
4,Poland,Przasnysz,34,Good,1,Good,34,Good,0,Good,20,Good,53.0167,20.8833


In [260]:
df.describe()

,AQI Value,CO AQI Value,Ozone AQI Value,NO2 AQI Value,PM2.5 AQI Value,lat,lng
count,16695.000000,16695.000000,16695.000000,16695.000000,16695.000000,16695.000000,16695.000000
mean,62.998682,1.342138,31.767355,3.819647,59.821324,30.267148,-3.944485
std,43.091971,2.371379,22.839343,5.880677,43.208298,22.947398,73.037148
min,7.000000,0.000000,0.000000,0.000000,0.000000,-54.801900,-171.750000
25%,38.500000,1.000000,20.000000,0.000000,34.000000,16.515450,-75.180000
50%,52.000000,1.000000,29.000000,2.000000,52.000000,38.815800,5.643100
75%,69.000000,1.000000,38.000000,5.000000,69.000000,46.683300,36.275000
max,500.000000,133.000000,222.000000,91.000000,500.000000,70.767000,178.017800


Here we are `defining` the `columns of interest` for our Machine Learning models

In [261]:
learning_vars = ['PM2.5 AQI Value', 'NO2 AQI Value', 'Ozone AQI Value', 'CO AQI Value']
target_classification = 'AQI Category'
target_regression = 'AQI Value'
fix_columns = ['PM2.5 AQI Value', 'NO2 AQI Value', 'Ozone AQI Value', 'CO AQI Value', 'AQI Value']

# Data Clean up

We are using `dropna()` since it provided us the best accuracy, and we `didn't lose out` on many datapoints that made much difference.

In [262]:
# imputer = SimpleImputer(strategy='mean')
# df[fix_columns] = imputer.fit_transform(df[fix_columns])
df = df.dropna()
# knn_imputer = KNNImputer(n_neighbors=5) 
# df[fix_columns] = knn_imputer.fit_transform(df[fix_columns])

In [263]:
df.head()

,Country,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category,lat,lng
0,Russian Federation,Praskoveya,51,Moderate,1,Good,36,Good,0,Good,51,Moderate,44.7444,44.2031
1,Brazil,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good,-5.2900,-44.4900
2,Brazil,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good,-11.2958,-41.9869
3,Italy,Priolo Gargallo,66,Moderate,1,Good,39,Good,2,Good,66,Moderate,37.1667,15.1833
4,Poland,Przasnysz,34,Good,1,Good,34,Good,0,Good,20,Good,53.0167,20.8833


We make use of `LabelEncoder()` to make it easier for the models to learn data. Since `models perform better` with `number` than `words`, for classification, we encode each class with a numerical value.

In [264]:
label_encoder = LabelEncoder()
df[target_classification] = label_encoder.fit_transform(df[target_classification])

In [265]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16393 entries, 0 to 16694
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Country             16393 non-null  object 
 1   City                16393 non-null  object 
 2   AQI Value           16393 non-null  int64  
 3   AQI Category        16393 non-null  int64  
 4   CO AQI Value        16393 non-null  int64  
 5   CO AQI Category     16393 non-null  object 
 6   Ozone AQI Value     16393 non-null  int64  
 7   Ozone AQI Category  16393 non-null  object 
 8   NO2 AQI Value       16393 non-null  int64  
 9   NO2 AQI Category    16393 non-null  object 
 10  PM2.5 AQI Value     16393 non-null  int64  
 11  PM2.5 AQI Category  16393 non-null  object 
 12  lat                 16393 non-null  float64
 13  lng                 16393 non-null  float64
dtypes: float64(2), int64(6), object(6)
memory usage: 1.9+ MB


In [266]:
df.head()

,Country,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category,lat,lng
0,Russian Federation,Praskoveya,51,2,1,Good,36,Good,0,Good,51,Moderate,44.7444,44.2031
1,Brazil,Presidente Dutra,41,0,1,Good,5,Good,1,Good,41,Good,-5.2900,-44.4900
2,Brazil,Presidente Dutra,41,0,1,Good,5,Good,1,Good,41,Good,-11.2958,-41.9869
3,Italy,Priolo Gargallo,66,2,1,Good,39,Good,2,Good,66,Moderate,37.1667,15.1833
4,Poland,Przasnysz,34,0,1,Good,34,Good,0,Good,20,Good,53.0167,20.8833


`Splitting` dataset into `train` and `test`. We will be using `80:20` ratio to perform the split.

In [267]:
X = df[learning_vars]

y_class = df[target_classification]
y_reg = df[target_regression]

X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X, y_class, test_size=0.2, random_state=0)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y_reg, test_size=0.2, random_state=0)

In [268]:
# smote = SMOTE(random_state=0)
# X_train_class, y_train_class = smote.fit_resample(X_train_class, y_train_class)

In [269]:
X.head()

,PM2.5 AQI Value,NO2 AQI Value,Ozone AQI Value,CO AQI Value
0,51,0,36,1
1,41,1,5,1
2,41,1,5,1
3,66,2,39,1
4,20,0,34,1


In [270]:
scaler = StandardScaler()
X_train_class = scaler.fit_transform(X_train_class)
X_test_class = scaler.transform(X_test_class)
X_train_reg = scaler.fit_transform(X_train_reg)
X_test_reg = scaler.transform(X_test_reg)

In [271]:
logistic_model = LogisticRegression(max_iter=2000)
logistic_model.fit(X_train_class, y_train_class)
y_pred_class_logistic = logistic_model.predict(X_test_class)
print("Logistic Regression Classification Report:")
print(classification_report(y_test_class, y_pred_class_logistic, zero_division=1))

Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.94      0.95      1479
           1       0.90      0.75      0.82        12
           2       0.93      0.94      0.94      1386
           3       0.87      0.98      0.92       187
           4       0.96      0.84      0.89       187
           5       0.79      0.54      0.64        28

    accuracy                           0.94      3279
   macro avg       0.90      0.83      0.86      3279
weighted avg       0.94      0.94      0.93      3279



In [272]:
dtree_classifier = DecisionTreeClassifier(random_state=0,max_depth=7,min_samples_split=70)
dtree_classifier.fit(X_train_class, y_train_class)
y_pred_class_dtree=dtree_classifier.predict(X_test_class)
print("Decision Tree Classification Report:")
print(classification_report(y_test_class, y_pred_class_dtree, zero_division=0))

Decision Tree Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1479
           1       1.00      0.92      0.96        12
           2       1.00      1.00      1.00      1386
           3       1.00      0.99      0.99       187
           4       0.99      1.00      0.99       187
           5       0.97      1.00      0.98        28

    accuracy                           1.00      3279
   macro avg       0.99      0.98      0.99      3279
weighted avg       1.00      1.00      1.00      3279



In [273]:
dtree_regressor = DecisionTreeRegressor(random_state=0,max_depth=10,min_samples_leaf=3,min_samples_split=90)
dtree_regressor.fit(X_train_reg, y_train_reg)
y_pred_reg_dt=dtree_regressor.predict(X_test_reg)
print("Decision Tree Regression Metrics:")
print("MSE:", mean_squared_error(y_test_reg, y_pred_reg_dt))
print("R2 Score:", r2_score(y_test_reg, y_pred_reg_dt))

Decision Tree Regression Metrics:
MSE: 43.28609663066803
R2 Score: 0.9772954732691793


In [274]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=0)
rf_classifier.fit(X_train_class, y_train_class)
y_pred_class_rf = rf_classifier.predict(X_test_class)
print("Random Forest Classification Report:")
print(classification_report(y_test_class, y_pred_class_rf, zero_division=1))

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1479
           1       1.00      0.92      0.96        12
           2       1.00      1.00      1.00      1386
           3       0.99      0.99      0.99       187
           4       1.00      0.99      1.00       187
           5       0.93      1.00      0.97        28

    accuracy                           1.00      3279
   macro avg       0.99      0.98      0.99      3279
weighted avg       1.00      1.00      1.00      3279



In [275]:
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train_reg, y_train_reg)
y_pred_reg_rf = rf_regressor.predict(X_test_reg)
print("Random Forest Regression Metrics:")
print("MSE:", mean_squared_error(y_test_reg, y_pred_reg_rf))
print("R2 Score:", r2_score(y_test_reg, y_pred_reg_rf))

Random Forest Regression Metrics:
MSE: 3.6444213784690436
R2 Score: 0.9980884193991472


In [276]:
xgb_classifier = xgb.XGBClassifier(random_state=0,max_depth=10,learning_rate=0.1,n_estimators= 100)
xgb_classifier.fit(X_train_class, y_train_class)
y_pred_class_xgb=xgb_classifier.predict(X_test_class)
print("XGBoost Classification Report:")
print(classification_report(y_test_class, y_pred_class_xgb))

XGBoost Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1479
           1       0.79      0.92      0.85        12
           2       1.00      1.00      1.00      1386
           3       0.97      0.96      0.96       187
           4       0.99      0.99      0.99       187
           5       0.76      0.79      0.77        28

    accuracy                           0.99      3279
   macro avg       0.92      0.94      0.93      3279
weighted avg       0.99      0.99      0.99      3279



In [277]:
xgb_regressor = xgb.XGBRegressor(random_state=0,max_depth=5,n_estimators= 100)
xgb_regressor.fit(X_train_reg, y_train_reg)
y_pred_reg_xgb=xgb_regressor.predict(X_test_reg)
print("XGBoost Regression Metrics:")
print("MSE:", mean_squared_error(y_test_reg, y_pred_reg_xgb))
print("R2 Score:", r2_score(y_test_reg, y_pred_reg_xgb))

XGBoost Regression Metrics:
MSE: 29.3255935050691
R2 Score: 0.984618067741394


In [278]:
mlp_classifier = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
mlp_classifier.fit(X_train_class, y_train_class)
y_pred_class_mlp = mlp_classifier.predict(X_test_class)
print("MLP Classification Report:")
print(classification_report(y_test_class, y_pred_class_mlp, zero_division=1))

MLP Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1479
           1       0.85      0.92      0.88        12
           2       1.00      0.99      1.00      1386
           3       0.95      0.99      0.97       187
           4       0.96      0.97      0.96       187
           5       0.88      0.79      0.83        28

    accuracy                           0.99      3279
   macro avg       0.94      0.94      0.94      3279
weighted avg       0.99      0.99      0.99      3279



In [279]:
mlp_regressor = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=300, random_state=42)
mlp_regressor.fit(X_train_reg, y_train_reg)
y_pred_reg_mlp = mlp_regressor.predict(X_test_reg)
print("MLP Regression Metrics:")
print("MSE:", mean_squared_error(y_test_reg, y_pred_reg_mlp))
print("R2 Score:", r2_score(y_test_reg, y_pred_reg_mlp))

MLP Regression Metrics:
MSE: 5.408030823180654
R2 Score: 0.9971633667633821


In [280]:
y_dual = df[[target_classification, target_regression]]
X_train_dual, X_test_dual, y_train_dual, y_test_dual = train_test_split(X, y_dual, test_size=0.2, random_state=42)

X_train_dual = scaler.fit_transform(X_train_dual)
X_test_dual = scaler.transform(X_test_dual)

mlp_dual = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
mlp_dual.fit(X_train_dual, y_train_dual)
y_pred_dual = mlp_dual.predict(X_test_dual)

print("MLP Dual Prediction Metrics:")
print("MSE (AQI_Bucket):", mean_squared_error(y_test_dual[target_classification], y_pred_dual[:, 0]))
print("MSE (AQI):", mean_squared_error(y_test_dual[target_regression], y_pred_dual[:, 1]))

MLP Dual Prediction Metrics:
MSE (AQI_Bucket): 0.23110280254635465
MSE (AQI): 4.5222558923905085


In [281]:
joblib.dump(logistic_model, 'logistic_model.pkl')
joblib.dump(dtree_classifier,'dtree_classifier.pkl')
joblib.dump(dtree_regressor,'dtree_regressor.pkl')
joblib.dump(rf_classifier, 'rf_classifier.pkl')
joblib.dump(rf_regressor, 'rf_regressor.pkl')
joblib.dump(xgb_classifier,'xgb_classifier.pkl')
joblib.dump(xgb_regressor,'xgb_regressor.pkl')
joblib.dump(mlp_classifier, 'mlp_classifier.pkl')
joblib.dump(mlp_regressor, 'mlp_regressor.pkl')
joblib.dump(mlp_dual, 'mlp_dual.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')

['label_encoder.pkl']

In [282]:
def predict_models(x_new_test):
    # Load models and scaler
    logistic_model = joblib.load('logistic_model.pkl')
    dtree_classifier = joblib.load('dtree_classifier.pkl')
    dtree_regressor = joblib.load('dtree_regressor.pkl')
    rf_classifier = joblib.load('rf_classifier.pkl')
    rf_regressor = joblib.load('rf_regressor.pkl')
    xgb_classifier = joblib.load('xgb_classifier.pkl')
    xgb_regressor = joblib.load('xgb_regressor.pkl')
    mlp_classifier = joblib.load('mlp_classifier.pkl')
    mlp_regressor = joblib.load('mlp_regressor.pkl')
    mlp_dual = joblib.load('mlp_dual.pkl')
    scaler = joblib.load('scaler.pkl')
    label_encoder = joblib.load('label_encoder.pkl')

    # Scale the custom data
    x_new_test_scaled = scaler.transform(x_new_test)

    # Make predictions
    logistic_pred = logistic_model.predict(x_new_test_scaled)
    logistic_label = label_encoder.inverse_transform(logistic_pred)

    dtree_class_pred = dtree_classifier.predict(x_new_test_scaled)
    dtree_class_label = label_encoder.inverse_transform(dtree_class_pred)

    dtree_reg_pred = dtree_regressor.predict(x_new_test_scaled)

    rf_class_pred = rf_classifier.predict(x_new_test_scaled)
    rf_class_label = label_encoder.inverse_transform(rf_class_pred)

    rf_reg_pred = rf_regressor.predict(x_new_test_scaled)

    xgb_class_pred = xgb_classifier.predict(x_new_test_scaled)
    xgb_class_label = label_encoder.inverse_transform(xgb_class_pred)

    xgb_reg_pred = xgb_regressor.predict(x_new_test_scaled)

    mlp_class_pred = mlp_classifier.predict(x_new_test_scaled)
    mlp_class_label = label_encoder.inverse_transform(mlp_class_pred)

    mlp_reg_pred = mlp_regressor.predict(x_new_test_scaled)

    mlp_dual_pred = mlp_dual.predict(x_new_test_scaled)
    mlp_dual_class_label = label_encoder.inverse_transform(mlp_dual_pred[:, 0].astype(int))
    mlp_dual_reg_pred = mlp_dual_pred[:, 1]

    # Print predictions
    print("Logistic Regression Prediction:", logistic_label)
    print("Decison Tree Classification Prediction:", dtree_class_label)
    print("Decision Tree Regression Prediction:", dtree_reg_pred)
    print("Random Forest Classification Prediction:", rf_class_label)
    print("Random Forest Regression Prediction:", rf_reg_pred)
    print("XGBoost Classification Prediction:", xgb_class_label)
    print("XGBoost Regression Prediction:", xgb_reg_pred)
    print("MLP Classification Prediction:", mlp_class_label)
    print("MLP Regression Prediction:", mlp_reg_pred)
    print("MLP Dual Prediction - Classification Label:", mlp_dual_class_label)
    print("MLP Dual Prediction - Regression Value:", mlp_dual_reg_pred)


# Testing with real-world data from Accuweather

### October 20, 2024

In [283]:
test_data_mangalore = {
    'PM2.5 AQI Value': [41],
    'NO2 AQI Value': [26],
    'Ozone AQI Value': [4],
    'CO AQI Value': [3]
}

mangalore_test = pd.DataFrame(test_data_mangalore)

In [284]:
test_data_kolkata = {
    'PM2.5 AQI Value': [180],
    'NO2 AQI Value': [54],
    'Ozone AQI Value': [1],
    'CO AQI Value': [7]
}
kolkata_test = pd.DataFrame(test_data_kolkata)

In [285]:
predict_models(mangalore_test)

Logistic Regression Prediction: ['Good']
Decison Tree Classification Prediction: ['Good']
Decision Tree Regression Prediction: [41.]
Random Forest Classification Prediction: ['Good']
Random Forest Regression Prediction: [41.]
XGBoost Classification Prediction: ['Good']
XGBoost Regression Prediction: [39.82113]
MLP Classification Prediction: ['Good']
MLP Regression Prediction: [42.76564618]
MLP Dual Prediction - Classification Label: ['Good']
MLP Dual Prediction - Regression Value: [43.49993401]


In [286]:
predict_models(kolkata_test)

Logistic Regression Prediction: ['Unhealthy']
Decison Tree Classification Prediction: ['Unhealthy']
Decision Tree Regression Prediction: [183.48837209]
Random Forest Classification Prediction: ['Unhealthy']
Random Forest Regression Prediction: [179.2]
XGBoost Classification Prediction: ['Unhealthy']
XGBoost Regression Prediction: [165.11871]
MLP Classification Prediction: ['Unhealthy']
MLP Regression Prediction: [180.05516051]
MLP Dual Prediction - Classification Label: ['Unhealthy for Sensitive Groups']
MLP Dual Prediction - Regression Value: [180.15520791]
